# Setup

In [1]:
import json
import pprint as pp
import requests
from requests.utils import requote_uri
import sqlite3
import datetime, time
import re

localDB='doi_prefixes.db'
doi_table_name = 'dois'
prefix_table_name = 'prefixes'
fld_prefix='prefix'
fld_count='cnt'
fld_org='org'
fld_file='file'
fld_doi='doi'


prefixes_datacite=[]
prefixes_crossref=[]
prefixes={}

# Load DataCite prefixes

In [2]:
def getDataCitePrefixes():
    urlString='https://api.datacite.org/prefixes'
    r = requests.get(urlString)
    data=r.json()['data']

    for d in data:
        prefixes_datacite.append(d['id'])

In [3]:
def markDataCitePrefixes():
    db = sqlite3.connect(localDB)
    cur = db.cursor()

    pList="'{0}'".format("', '".join(prefixes_datacite))
    sql="UPDATE {tn} SET {nf1}='{val1}' WHERE {nf2} in ({val2})"\
        .format(tn=prefix_table_name, nf1=fld_org,nf2=fld_prefix,\
        val1='DataCite',val2=pList)
    print(sql)
    cur.execute(sql)        
    db.commit()
    db.close()
    
#getDataCitePrefixes
#markDataCitePrefixes()

# Load Scholix Prefixes from SqliteDB

In [4]:
def getPrefixOrg(prefix):
    urlString='https://api.crossref.org/prefixes/{}'.format(prefix)
    r = requests.get(urlString)
    if r.status_code == 200: 
        return r.json()['message']['name']
    else:
        urlString='https://api.datacite.org/prefixes/{}'.format(prefix)
        r = requests.get(urlString)
        if r.status_code == 200: 
            org=r.json()['data']['attributes']['registration-agency']
            for i in r.json()['data']['relationships']['providers']['data']:
                org='{}:{}'.format(org,i['id'])
            return org
        else:
            return None

#Test        
#print(checkCrossref('10.2912'))
#print(checkCrossref('10.1371'))

In [5]:
def matchPrefixReg (prefix):
    if len(prefix.strip())>0:
        match=re.search( r'(10\.\d{4,5})', prefix, re.M|re.I)
        if match:
            return (match.group())
        else:
            print ('Invalid Prefix: {}'.format(prefix))
            return None

In [ ]:
db = sqlite3.connect(localDB)
c = db.cursor() 
sql="Select * from prefixes where org='-'"
c.execute(sql)
all_rows = c.fetchall()
print('Processing {} prefixes'.format(len(all_rows)))
i=0
for r in all_rows:
    i=i+1
    if i>10: break
    prefix=matchPrefixReg(r[0])
    if prefix:
        org=getPrefixOrg(prefix)
        print(sql)        
        sql='update prefixes Set org="{}" where prefix="{}"'.format(org,r[0])
        c.execute(sql)
    else:
        sql='update prefixes Set org="X" where prefix="{}"'.format(r[0])
        print(sql)
        c.execute(sql)
db.commit()
#db.rollback()
db.close()

Processing 1920 prefixes
